In [51]:
from datasets import load_dataset
dataset = load_dataset("Cartinoe5930/raw_text_synthetic_dataset_50k", split = "train")

# 역색인

In [52]:
from kiwipiepy import Kiwi
from collections import defaultdict
import math

In [53]:
class InvertedIndex:
    def __init__(self):
        self.reset()

    def reset(self):
        self.index = defaultdict(dict)
        self.kiwi = Kiwi()
        self.document_lengths = {}
        self.total_documents = 0
        self.average_document_length = 0
        self.documents = {}

    def tokenize(self, text):
        return [token.form for token in self.kiwi.tokenize(text)]

    def add_document(self, doc_id, question, answer):
        tokens = self.tokenize(question)
        self.document_lengths[doc_id] = len(tokens)
        self.total_documents += 1
        self.documents[doc_id] = {'question': question, 'answer': answer}

        for token in set(tokens):
            if doc_id not in self.index[token]:
                self.index[token][doc_id] = 0
            self.index[token][doc_id] += tokens.count(token)

        self.average_document_length = sum(self.document_lengths.values()) / self.total_documents

    def calculate_bm25_score(self, query_tokens, doc_id):
        k1 = 1.5
        b = 0.75
        score = 0

        for token in query_tokens:
            if token not in self.index or doc_id not in self.index[token]:
                continue

            tf = self.index[token][doc_id]
            df = len(self.index[token])
            idf = math.log((self.total_documents - df + 0.5) / (df + 0.5) + 1)

            numerator = tf * (k1 + 1)
            denominator = tf + k1 * (1 - b + b * self.document_lengths[doc_id] / self.average_document_length)
            score += idf * numerator / denominator

        return score

    def search(self, query, k=5):
        query_tokens = self.tokenize(query)
        scores = defaultdict(float)

        for token in query_tokens:
            if token in self.index:
                for doc_id in self.index[token]:
                    scores[doc_id] += self.calculate_bm25_score(query_tokens, doc_id)

        top_k = sorted(scores.items(), key=lambda x: x[1], reverse = True)[:k]
        return [(doc_id, score, self.documents[doc_id]) for doc_id, score in top_k]

In [54]:
# add docs
from tqdm import tqdm

index = InvertedIndex()
for idx, data in enumerate(tqdm(dataset)):
    question = data['question']
    answer = data['response']
    index.add_document(idx, question, answer)

100%|██████████| 21900/21900 [00:48<00:00, 454.25it/s]


# 비슷한 문제들을 참고해서 만들게 하자

In [55]:
print(type(dataset))
dataset = dataset.select(range(625))
print(type(dataset))
'''
dataset2 = dataset[5000:10000] # 동재
dataset3 = dataset[10000:15000] # 기홍
dataset4 = dataset[15000:20000] # 경도
dataset5 = dataset[20000:]
'''

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


'\ndataset2 = dataset[5000:10000] # 동재\ndataset3 = dataset[10000:15000] # 기홍\ndataset4 = dataset[15000:20000] # 경도\ndataset5 = dataset[20000:]\n'

In [56]:
import openai
from dotenv import load_dotenv
import os

dotenv_path = os.path.join(os.getcwd(), '.env')
load_dotenv(dotenv_path)

openai_key = os.getenv('OPENAI_TEAM_API_KEY')
client = openai.OpenAI(api_key=openai_key)

In [57]:
system_prompt = """
You are given two pairs of reference questions and reference answers.
Your role is a questioner who make a new question.
When making your questions, consider the following.
1. New Question must require choices such as 'Which is right', 'Which is not right', 'Which is most appropriate', and 'Which is not most appropriate'.
2. You have to make 5 choices, 1 answer choice and 4 wrong choices.
3. The choices must be generated in association with one of several keywords in the reference question and answer.
4. The wrong answer and the right answer are confused, but the wrong answer must be a clear wrong answer.
5. The choices does not deviate from the subject of the problem, but it must be different.
6. The choices sentence must be similar in length.
7. If a person is a financial expert, the person can solve the problem, but if the person is a beginner in financial knowledge, please make the problem with a difficulty that the person cannot solve because it is difficult.
8. Please don't create a problem that can be solved by reading other than the problem.
please write in Korean and you must write the answer on the last line.
"""

user_prompt ="""
### Reference
### Question 1: {}
### Answer 1: {}

### Question 2: {}
### Answer 2: {}

### New Question : 

"""


In [58]:
import random
from tqdm import tqdm

new_questions = []
for idx, data in enumerate(tqdm(dataset, total=len(dataset))):
    question = data['question']
    answer = data['response']
    system_msg = {'role':'system', 'content': system_prompt}

    similar_questions = index.search(question, k=2)

    user_msg = {'role':'user', 'content': user_prompt.format(
        similar_questions[0][2]['question'],
        similar_questions[0][2]['answer'],
        similar_questions[1][2]['question'],
        similar_questions[1][2]['answer'])}

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages = [system_msg, user_msg],
    )

    result = response.choices[0].message.content
    new_questions.append(result)
    
    if idx % 100 == 0:
        print(result)
        print("*"*100)


  0%|          | 0/625 [00:00<?, ?it/s]

  0%|          | 1/625 [00:26<4:38:28, 26.78s/it]

AI 시스템이 금융 서비스에서 의사결정을 내릴 때, 윤리적 고려사항과 투명성을 아우르기 위해 어떤 조치를 취하는 것이 가장 적절한가요? 

다음 중 가장 적절한 방안은 무엇인가요?

1. **고객 피드백을 반영해 AI의 학습 데이터를 정기적으로 수정하는 것**  
2. **AI 시스템의 의사결정에 대한 설명을 제공하지 않는 것**  
3. **윤리적 기준 없이 성과 목표만 중시하는 것**  
4. **전문가와의 협의 없이 혼자서 데이터 결정을 내리는 것**  
5. **데이터 품질 개선을 위한 외부 데이터를 활용하는 것**  

정답은 1번입니다.
****************************************************************************************************


 16%|█▌        | 101/625 [30:20<3:19:06, 22.80s/it]

AI 시스템의 공정성, 투명성 및 설명 가능성을 보장하기 위한 금융 서비스 조직의 정책 및 절차 중에서 가장 적절한 것은 무엇인가요?  

1. 정책 개발 시 AI 의사결정 모델의 검토 및 평가 절차를 포함해야 한다.  
2. 교육 프로그램을 생략하고, 개발자와 사용자들에게 별도로 교육할 필요가 없다.  
3. 데이터 관리 체계에서 편향을 최소화하기 위한 기준을 마련해야 한다.  
4. 고객 피드백은 반영하지 않고, 내부 평가만 필요하다.  
5. AI 시스템은 사용자가 이해할 수 없는 방식으로 작동해야 한다.  

정답: 1. 정책 개발 시 AI 의사결정 모델의 검토 및 평가 절차를 포함해야 한다.
****************************************************************************************************


...

 80%|████████  | 501/625 [2:21:51<35:17, 17.08s/it]  

해결 기금의 관리에서 가장 적절한 요소는 무엇이며, 해당 요소가 은행 안정성에 어떻게 영향을 미치는지 설명하세요.

1. 자본의 적정성은 은행의 리스크 관리에 핵심적인 역할을 합니다.
2. 자본 구조의 다양성은 은행 안정성에 부정적인 영향을 미칩니다.
3. 자본의 적정성은 외부의 심리적 안전감을 저해합니다.
4. 자본의 과잉 적정성은 금융 취약성을 증가시킵니다.
5. 자본의 적정성은 고객의 예금을 보호하는 데 직접적인 기여를 합니다.

정답: 1.
****************************************************************************************************


 96%|█████████▌| 601/625 [2:46:57<05:24, 13.51s/it]

금융 소비자가 기만적인 금융 관행으로부터 보호받기 위해서는 어떤 정책이 가장 적절한가? 다음 중 가장 적절한 정책은 무엇인가?

1. 금융 기관의 영업 비밀 보호를 강화하는 정책
2. 소비자에게 금융 상품 정보 제공을 의무화하는 정책
3. 금융 소비자의 불만을 공개적으로 처리하는 정책
4. 금융 교육을 제공하지 않는 정책
5. 금융 상품의 가격 인상을 유도하는 정책

정답: 2. 소비자에게 금융 상품 정보 제공을 의무화하는 정책
****************************************************************************************************


100%|██████████| 625/625 [2:52:59<00:00, 16.61s/it]


# CSV 파일로 만들기

In [59]:
mcqa_data = []

for raw_question in new_questions:
    lines = [line.strip() for line in raw_question.split('\n') if line.strip() != '']
    question = lines[:-1]
    answer = lines[-1]
    
    for i in range(6):
        if str(i) in answer:
            answer = i
            break

    item = {"question": question, 'answer': answer}
    mcqa_data.append(item)

In [61]:
import pandas as pd
result_df = pd.DataFrame(mcqa_data)
result_df.to_csv('/root/KRX_LLM/data/mcqa_data1.csv', index=False)